In [1]:
import pandas as pd 
import yaml
import notebook_utils as nu

In [2]:
#load the files
with open('jobs_parsing.yaml') as f:
    conf_dict = yaml.safe_load(f)
language_list = nu.read_csv_to_list (conf_dict['LANGUAGE_PATH'])
df = pd.read_csv(conf_dict['TABLE_PATH'], usecols = conf_dict['COLS'].keys())

In [ ]:
n_cols =df.shape[1]
df['header.jobTitle'] = df['header.jobTitle'].map(lambda x: x.lower())
df.insert(n_cols, 'IT_job', df['header.jobTitle'].str.contains('|'.join(conf_dict['TECH_KEYWORDS'])))
df.insert(n_cols, 'entry_job', ~df['header.jobTitle'].str.contains('|'.join(conf_dict['SENIOR_KEYWORDS'])))
df.insert(n_cols, 'Swiss_based', df["map.country"].map(lambda x: x in conf_dict['SWISS_LOCATION']))

df_Swiss_IT  = df.loc[(df['Swiss_based'] ==True) & (df['IT_job'] == True) & (df['entry_job'] == True)]
df_Swiss_IT.insert(n_cols, 'job_description', df_Swiss_IT['job.description'].map(lambda x: clean_from_html(str(x))))
df_Swiss_IT.insert(n_cols, 'english', df_Swiss_IT["job_description"].map(lambda x: nu.if_en(str(x))))
df_Swiss_IT_en = df_Swiss_IT.loc[df_Swiss_IT['english'] ==True]
df_Swiss_IT_en.drop(columns=['english', 'job.description', 'IT_job', 'Swiss_based', 'entry_job'], inplace = True)
df_Swiss_IT_en.reset_index(drop = True, inplace = True)
df_Swiss_IT_en['skills'] = df_Swiss_IT_en["job_description"].map(lambda x: nu.extract_skills(str(x), language_list))
df_Swiss_IT_en['soft_skills'] = df_Swiss_IT_en['skills'].map(lambda x: x[0])
df_Swiss_IT_en['hard_skills'] = df_Swiss_IT_en['skills'].map(lambda x: x[1])
df_Swiss_IT_en['languages'] = df_Swiss_IT_en['skills'].map(lambda x: x[2])

df_Swiss_IT_en.rename(mapper = conf_dict['COLS'], axis = 1, inplace = True)
df_Swiss_IT_en.drop(columns = ['skills'], inplace = True)
df_Swiss_IT_en.to_csv('IT_entry_swiss_jobs.csv')

import csv
fields = ['langugages']
with open('languages.csv', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    for item in data_into_list:
        write.writerow([item])